In [1]:
from bs4 import BeautifulSoup
import requests

In [ ]:
import bs4

print(bs4.__version__)
print(requests.__version__)

# Empezamos el scraping

## 1) Obtener el HTML

In [4]:
URL_BASE = 'https://scrapepark.org/spanish/'
pedido_obtenido = requests.get(URL_BASE)
html_obtenido = pedido_obtenido.text

## 2) Parsear el HTML

In [5]:
soup = BeautifulSoup(html_obtenido,"html.parser")

## El método find() nos permite quedarnos con la información asociada a una etiqueta de HTML

In [ ]:
primer_h2 = soup.find('h2')
print(primer_h2)
print(primer_h2.text)

## El método find_all() busca todos los elementos de la página con esa etiqueta y devuelve una "lista" que los contiene (en realidad devuelve un objeto de la clase bs4.element.ResultSet).

In [ ]:
h2_todos=soup.find_all('h2')
print(h2_todos)

## Si usamos el parametro limit=1, emulamos al método find

In [ ]:
h2_uno_solo = soup.find_all('h2',limit=1)
print(h2_uno_solo)

## Podemos iterar sobre el objeto

In [ ]:
h2=soup.find_all('h2')
for i in h2:
    print(i.text)

## get_text() para más funcionalidades

In [ ]:
for seccion in h2:
    print(seccion.get_text(strip=True))

## Utilizando atributos de las etiquetas

## Clase